<a href="https://colab.research.google.com/github/lsteffenel/CHPS0704/blob/main/TP3/5_DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a id='chap-tpdbscan'></a>

# Travaux pratiques - DBSCAN



Références externes utiles :

> - [Documentation NumPy](https://docs.scipy.org/doc/numpy/user/index.html)  
- [Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- [Documentation MatPlotLib](http://matplotlib.org/)  
- [Site scikit-learn](http://scikit-learn.org/stable/index.html)  
- [Site langage python](https://www.python.org)  



L’objectif de cette séance de TP est d’illustrer la mise en pratique de
la classification automatique avec DBSCAN. Dans un premier temps, nous
observerons le comportement de l’algorithme dans un cas synthétique
mettant en échec *k-means*. Nous verrons comment utiliser les
heuristiques classiques pour déterminer les paramètres de l’algorithme
et comment interpréter le partitionnement obtenu. Enfin, nous
appliquerons DBSCAN sur un jeu de données réelles et nous illustrerons
son usage pour la détection de données aberrantes.

## La classification par densité dans scikit-learn

Comme nous l’avons vu dans le TP précédent, scikit-learn propose de
nombreuses [méthodes de classification
automatique](http://scikit-learn.org/stable/modules/clustering.html).
Nous nous intéressons pour cette séance à **DBSCAN** (*Density-Based
Spatial Clustering of Applications with Noise*).

Comme *k-means*, DBSCAN permet d’obtenir pour une matrice d’observation
`n_samples x n_features` un vecteur correspondant aux `n_samples`
numéros des groupes (*cluster*) pour ces observations. En revanche,
DBSCAN est une méthode transductive : l’algorithme ne permet pas de
produire des classifications pour de nouvelles données (il est
nécessaire de réentraîner le modèle en incluant les nouvelles
observations).

La description de l’implémentation de DBSCAN se trouve dans
[https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

## Classification avec DBSCAN de données générées

Commençons par générer un jeu de données de 500 observations dans un
espace 2D. Nous allons utiliser une fonction intégrée à scikit-learn
permettant de produire des nuages de points circulaires
([make_circles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html)).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
from sklearn.utils import shuffle

# Générons un nuage de points composé de deux cercles
# Le nuage contient 500 observations (`n_samples`) bruitées par
# un bruit gaussien d'écart-type 0.1 (`noise`).
# Le rapport entre le rayon du petit cercle et du grand cercle
# est de 0.3 (`factor`).
data, labels = sklearn.datasets.make_circles(n_samples=500, noise=0.1, factor=0.3, random_state=0)

print(data.shape)
# Permutation aléatatoire des lignes de la matrice (on mélange les observations)
data, labels = shuffle(data, labels)

# Affichage du nuage de points
plt.scatter(data[:,0], data[:,1], c=labels)
plt.show()

## Question :

Combien de groupes comporte ce jeu de données ?

## Question :

Réaliser un partitionnement de ce jeu de données à l’aide de $ k $-means. À quoi peut-on s’attendre ? Que constatez-vous ?

Les deux cercles étant séparés par une zone sans données, une méthode
basée sur la densité semble appropriée. Nous pouvons créer un modèle de
*clustering* utilisant DBSCAN en l’important depuis scikit-learn :

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN()

Les arguments du constructeur de DBSCAN sont les suivants :

- `eps`: la dimension du voisinage, c’est-à-dire la distance maximale
  entre deux observations permettant de les considérer comme voisines
  l’une de l’autre,  
- `min_samples`: le nombre minimal de voisins que doit avoir un point
  central,  
- `metric`: la distance à considérer (par défaut, la distance
  Euclidienne est utilisée).  


Il est possible d’appeler les méthodes suivantes :

- `.fit(X)`: réalise une classification automatique en utilisant la
  méthode DBSCAN sur la matrice d’observations `X`. Les résultats
  sont stockés dans l’attribut `.labels_`.  
- `.fit_predict(X)`: identique à `.fit(X)` mais renvoie directement
  les étiquettes des groupes.  


Les attributs suivants sont disponibles après l’appel à la méthode
`.fit()`:

- `core_sample_indices_`: les indices des points centraux.  
- `labels_`: les numéros de groupe des points de la matrice
  d’observations.  

## Question :

Quelles sont les valeurs par défaut des paramètres importants de DBSCAN dans scikit-learn ($ \varepsilon $ et $ m $) ?

Appliquons une classification automatique par DBSCAN sur notre jeu de
données. Comme pour k-means, il est possible de réaliser cette étape en
deux temps en appelant `fit()` puis en accédant à l’attribut
`labels_`, ou de tout faire en une seule manipulation à l’aide de la
méthode `fit_predict()`:

In [ ]:
predictions = db.fit_predict(data)
# équivalent à
# db.fit(data)
# predictions = db.labels_

# Affichage du nuage de points coloré par les prédictions
plt.scatter(data[:,0], data[:,1], c=predictions)
plt.show()

## Question :

Que constatez-vous ? Sur quel paramètre est-il vraisemblablement nécessaire de jouer pour améliorer ce résultat ?

Pour raffiner notre analyse, nous allons appliquer l’heuristique de
Schubert, qui exploite le graphe des $ k $-distances dans le nuage
des observations. On considère pour l’instant que `min_samples` est
fixé à sa valeur par défaut, c’est-à-dire 5. Nous devons donc tracer les
graphe des 4-distances pour notre matrice d’observations.

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=4).fit(data)
distances, _ = nn.kneighbors(data)

## Question :

En vous aidant de la [documentation de NearestNeighbours dans scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors.kneighbors), expliquez ce que fait le code ci-dessus.

Nous pouvons maintenant tracer le graphe des 4-distances. Pour ce faire,
on ne conserve que la distance de chaque point à son quatrième voisin,
puis on trie cette liste par ordre décroissant.

In [ ]:
distances_triees = np.sort(distances[:,-1])[::-1]
plt.plot(distances_triees)
plt.xlabel("Nombre de points")
plt.ylabel("$4$-distance")
plt.show()

## Question :

À partir du graphe des 4-distances, déterminez la valeur de `eps` adaptée pour ce jeu de données en utilisant l’heuristique vue en cours. Appliquez de nouveau DBSCAN avec ces paramètres. Affichez le nuage de points obtenu.

## Question :

Combien obtenez vous de groupes ? À quoi correspondent les observations d’étiquette `-1` ?

## Classification avec DBSCAN des données « Iris »

Le jeu de données Iris est est un grand classique de l’apprentissage
statistique. Il comporte 150 observations de plantes selon quatre
attributs :

- longueur de la
  [sépale](https://fr.wikipedia.org/wiki/S%C3%A9pale),  
- largeur de la sépale,  
- longueur de la pétale,  
- largeur de la pétale.  


Les observations appartiennent à l’une des trois classes, correspondant
aux trois espèces d’iris: *Setosa*, *Versicolour* ou *Virginica*.

Plus de détails peuvent se trouver dans la
[documentation](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-dataset).

Iris est intégré à scikit-learn et est disponible depuis le sous-module
`sklearn.datasets`:

In [ ]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

## Question

Combien ce jeu de données contient-il d’observations?

Afin de simuler la présence de données aberrantes, nous allons générer aléatoirement 20 points bruitées, tirées selon une loi uniforme entre les valeurs minimales et maximales de chaque colonne de la matrice d’observation :

In [ ]:
min_, max_ = X.min(axis=0), X.max(axis=0)
noise = np.random.rand(20, 4) * (max_ - min_) + min_
X = np.concatenate((X, noise))
y = np.concatenate((y, -1 * np.ones(20, dtype=int)))

## Question :

Réalisez une analyse en composantes principales et visualisez le jeu de données Iris projeté selon ses deux premiers axes principaux.

## Question :

Appliquez une classification automatique à l’aide de DBSCAN (travaillez sur les données en dimension 4, pas les données projetées !). Visualisez les groupes obtenus dans le plan principal. Comparez ce résultat au partitionnement obtenu à l’aide d’un *k-means*.

Les points correpsondant à l’étiquette -1 sont ceux ayant été identifiés
comme données aberrantes (*outliers*). Si la valeur de
$ \varepsilon $ a bien été choisie, les observations bruitées que
nous avons injecté dans le jeu de données devraient ainsi avoir été
isolées par DBSCAN.

## Question :

À l’aide des fonctions de `sklearn.metrics`, calculez le taux de bonne détection des *outliers*. Jusqu’à quelle proportion de données bruitées le partitionnement obtenu par DBSCAN est-il robuste ?